# This tutorial will show you how to find the suitable habitat range for Bristlecone pine using GeoPySpark in a GeoNotebook

### FYI: You are currently inside of a GeoNotebook.
GeoNotebook is an application that provides client/server enviroment with inteactive visualization and analysis capabilities using Jupyter notebook, GeoJS and other open source tools.
It was developed jointly developed by [Kitware](http://www.kitware.com/) and [NASA Ames](https://www.nasa.gov/centers/ames/home/index.html).

This tutorial will focus on GeoPySpark functionality, but you can find more resources and tutorials about GeoNotebooks [here](https://github.com/OpenGeoscience/geonotebook/tree/master/notebooks)


### Suitability analysis is a classic GIS case study that enables the combination of factors to return a desired result 
This tutorial sets the premise that you are interested in two factors for locating Bristlecone pines:
- Located between 3,000 and 4,000 meters
- Located on a south facing slope
     

In [1]:
# GeoPySpark has lots of imports:
from pyspark import SparkContext
from geopyspark import geopyspark_conf
from geopyspark.geotrellis.constants import LayerType, LayoutScheme
from geopyspark.geotrellis.catalog import read, read_value, query, write
from geopyspark.geotrellis import Extent
from geopyspark.geotrellis.geotiff import get

You will need to set up a spark context. To learn more about what that means take a look [here](https://spark.apache.org/docs/latest/programming-guide.html#initializing-spark)

In [2]:
conf=geopyspark_conf(appName="BristleConePine")
conf.set('spark.ui.enabled', True)
sc = SparkContext(conf = conf)

Retrieving an elevation .tif from the local file system:

In [3]:
elev_rdd = get(
    pysc=sc, 
    layer_type='spatial', 
    uri='elevation.tif', max_tile_size=256, num_partitions=20)

Tile, reproject, pyramid:

In [10]:
elev_tiled_rdd = elev_rdd.to_tiled_layer()
elev_reprojected_rdd = elev_tiled_rdd.reproject(target_crs=3857, scheme="zoom").cache()
#elev_pyramided_rdd = elev_reprojected_rdd.pyramid_non_power_of_two(
#    start_zoom=8, end_zoom=1, col_power=2^8, row_power=2^8
#).cache()

elev_pyramided_rdd = elev_reprojected_rdd.pyramid(
    end_zoom=1
).cache()


In [16]:
elev_reprojected_rdd.zoom_level

11

Imports for creating a TMS server capable of serving layers with custom colormaps

In [5]:
from geopyspark.geotrellis.color import ColorMap
from geopyspark.geotrellis.tms import TMSServer
from geonotebook.wrappers import TMSRasterData

In [11]:
from geopyspark.geotrellis.color import get_colors_from_matplotlib
elev_histo        = elev_pyramided_rdd.get_histogram()
elev_colors       = get_colors_from_matplotlib('viridis', 100)
elev_color_map    = ColorMap.from_histogram(sc, elev_histo, elev_colors)

In [14]:
elev_tms = TMSServer.rdd_tms_server(
    sc, elev_pyramided_rdd, elev_color_map)

In [9]:
M.set_center(-118, 38, 6)

In [15]:
M.add_layer(TMSRasterData(elev_tms))

Added TMS server at host 0:0:0:0:0:0:0:0
Added TMS server at port 57702


In [13]:
M.remove_layer(M.layers[0])

IndexError: list index out of range

Classify the elevation such that values of interest (between 3,000 and 4,000 meters) return a value of 1.

In [8]:
# use: elev_reprojected_rdd
elev_reclass_pre = elev_tiled_rdd.reclassify({1000:2, 2000:2, 3000:2, 4000:1, 5000:2}, int)
elev_reclass_rdd = elev_reclass_pre.reclassify({1:1}, int)

In [9]:
elev_reclass_reprojected_rdd   = elev_reclass_pre.reproject(target_crs=3857, scheme="zoom")
elev_reclass_pyramid_rdd = elev_reclass_reprojected_rdd.pyramid(start_zoom=10, end_zoom=1)

In [10]:
elev_reclass_histo = elev_reclass_pyramid_rdd.get_histogram()

In [11]:
#elev_reclass_color_map = ColorMap.from_histogram(sc, elev_reclass_histo, get_breaks(sc, 'Viridis', num_colors=100))
elev_reclass_color_map = ColorMap.from_colors(
    pysc = sc,
    breaks =[1], 
    color_list = [0xff000080])

In [12]:
elev_reclass_tms = TMSServer.rdd_tms_server(sc, elev_reclass_pyramid_rdd, elev_reclass_color_map)

In [13]:
M.add_layer(TMSRasterData(elev_reclass_tms))

Added TMS server at host 0:0:0:0:0:0:0:0
Added TMS server at port 54606


In [22]:
M.remove_layer(M.layers[0])

Focal operation: aspect. To find south facing slopes

In [14]:
from geopyspark.geotrellis.neighborhood import Square
from geopyspark.geotrellis.constants import Operation, Neighborhood

In [34]:
elev_tiled_rdd.srdd.focal(
    Operation.ASPECT.value, 
    'square', 1.0, 0.0, 0.0).rdd().count()

99

In [17]:
# square_neighborhood = Square(extent=1)
aspect_rdd = elev_tiled_rdd.focal(
    Operation.SLOPE, 
    Neighborhood.SQUARE, 1)

Py4JJavaError: An error occurred while calling o34.focal.
: scala.MatchError: Square (of class java.lang.String)
	at geopyspark.geotrellis.GeoTrellisUtils$.getNeighborhood(GeoTrellisUtils.scala:54)
	at geopyspark.geotrellis.SpatialTiledRasterRDD.focal(TiledRasterRDD.scala:432)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)


In [23]:
aspect_reprojected_rdd   = aspect_rdd.reproject(target_crs=3857, scheme="zoom")
aspect_pyramid_rdd       = aspect_reprojected_rdd.pyramid(start_zoom=10, end_zoom=1)

In [24]:
aspect_histo        = aspect_pyramid_rdd.get_histogram()
aspect_color_map    = ColorMap.from_histogram(sc, aspect_histo, get_breaks(sc, 'Viridis', num_colors=256))
aspect_tms          = rdd_tms_server(sc, aspect_pyramid_rdd, aspect_color_map)

In [25]:
M.add_layer(TMSRasterData(aspect_tms))

Added TMS server at host 0:0:0:0:0:0:0:0
Added TMS server at port 58912


In [26]:
M.remove_layer(M.layers[0])

Reclassify values such that values between 120 and 240 degrees (south) have a value of 1

In [27]:
aspect_reclass_pre  = aspect_rdd.reclassify({120:2, 240:1, 360: 2}, int)
aspect_reclass      = aspect_reclass_pre.reclassify({1:1}, int)

In [28]:
aspect_reclass_reprojected_rdd   = aspect_reclass.reproject(target_crs=3857, scheme="zoom")
aspect_reclass_pyramid_rdd       = aspect_reclass_reprojected_rdd.pyramid(start_zoom=10, end_zoom=1)

In [29]:
aspect_reclass_histo       = aspect_reclass_pyramid_rdd.get_histogram()
aspect_reclass_color_map   = ColorMap.from_histogram(sc, aspect_reclass_histo, get_breaks(sc, 'Viridis', num_colors=256))
aspect_reclass_tms         = rdd_tms_server(sc, aspect_reclass_pyramid_rdd, aspect_reclass_color_map)

In [30]:
M.add_layer(TMSRasterData(aspect_reclass_tms))

Added TMS server at host 0:0:0:0:0:0:0:0
Added TMS server at port 55492


In [31]:
M.remove_layer(M.layers[0])

Now add the values togehter to find the suitable range:

In [32]:
added = elev_reclass_pyramid_rdd + aspect_reclass_pyramid_rdd

In [33]:
added_histo = added.get_histogram()
added_color_map = ColorMap.from_histogram(sc, added_histo, get_breaks(sc, 'Viridis', num_colors=256))
added_tms = rdd_tms_server(sc, added, added_color_map)

In [34]:
M.add_layer(TMSRasterData(added_tms))

Added TMS server at host 0:0:0:0:0:0:0:0
Added TMS server at port 52587


In [ ]:
added_pyramided_rdd = added.pyramid(start_zoom=10, end_zoom=1)

In [ ]:
added_histo = added_pyramided_rdd.get_histogram()

In [ ]:
color_map = ColorMap.from_histogram(sc, added_histo, get_breaks(sc, 'Inferno', num_colors=256)) 

In [ ]:
added_tms = rdd_tms_server(sc,added_pyramided_rdd , color_map)

In [ ]:
M.add_layer(TMSRasterData(added_tms))

In [ ]:
aspect_histo = aspect_pyramided_rdd.get_histogram()

In [ ]:
color_map = ColorMap.from_histogram(sc, aspect_histo, get_breaks(sc, 'Inferno', num_colors=256)) 

In [ ]:
aspect_tms = rdd_tms_server(sc, aspect_pyramided_rdd, color_map)

In [ ]:
M.add_layer(TMSRasterData(aspect_tms))

In [ ]:
histo.min_max()

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
v = elev_reprojected_rdd.lookup(342,787)
plt.imshow(v[0]['data'][0])

In [ ]:
M.add_layer(RddRasterData(PngRDD(elev_pyramided_rdd, "plasma"), name="elevation"))

In [ ]:
from geopyspark.geotrellis.neighborhoods import Square
from geopyspark.geotrellis.constants import SLOPE, ASPECT

In [ ]:
square_neighborhood = Square(extent=1)
slope_rdd = elev_tiled_rdd.focal(operation=ASPECT, neighborhood=square_neighborhood)

In [ ]:
slope_reprojected_rdd = slope_rdd.reproject(3857, scheme=ZOOM)
slope_pyramided_rdd = slope_reprojected_rdd.pyramid(start_zoom=10, end_zoom=1)

In [ ]:
M.add_layer(RddRasterData(PngRDD(slope_pyramided_rdd, "plasma"), name="elevation"))


In [ ]:
M.remove_layer(M.layers[0])